In [1]:
import os
# CUDAVISIBLE DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6"

os.environ['HF_DATASETS_CACHE'] = "/work/tianle/huggingface/datasets"
os.environ['HF_HOME'] = "/work/tianle/huggingface"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import transformers

/home/tianle/miniconda3/envs/code_attn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from snapkv.monkeypatch.monkeypatch import replace_llama, replace_mistral, replace_mixtral

In [3]:
replace_mixtral()

/work/data/tianle/share/SnapKV/snapkv/monkeypatch/monkeypatch.py:50: UserWarning: Transformers version 4.36.2 might not be compatible with SnapKV. SnapKV is tested with Transformers version ['4.37'].
  warnings.warn(f"Transformers version {transformers_version} might not be compatible with SnapKV. SnapKV is tested with Transformers version {version_list}.")


In [4]:
from fastchat.model import load_model, get_conversation_template


In [5]:
from importlib.metadata import version
try:
    transformers_version = version("transformers")
    print(f"Transformers version: {transformers_version}")
except Exception as e:
    print(f"Error: {e}")


Transformers version: 4.36.2


In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
    use_flash_attention_2=True
    )

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
Loading checkpoint shards: 100%|██████████| 19/19 [01:10<00:00,  3.71s/it]


In [7]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [8]:
# load './snapkv.txt'
with open('snapkv.txt', 'r') as f:
    content = f.read().strip()

In [9]:
question = "\n What is the repository of SnapKV?"

In [10]:
conv = get_conversation_template("longchat")
conv.messages = []
conv.append_message(conv.roles[0],content + question)
# conv.append_message(conv.roles[0],"Who is Kobe Bryant?")
conv.append_message(conv.roles[1], None)

In [11]:
prompt = conv.get_prompt()

In [12]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [13]:
input_ids_len = input_ids.size(1)

In [14]:
outputs = model.generate(input_ids.cuda(), max_new_tokens=200, do_sample=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [15]:
print(tokenizer.decode(outputs[0][input_ids_len:], skip_special_tokens=True))

The repository of SnapKV is available at <https://github.com/FasterDecoding/SnapKV>.
